In [18]:
import dlc_practical_prologue
import torch
from torch import nn
from torch import optim
import torchvision
from torch.nn import functional as F
from tqdm import trange

In [2]:
train_input, train_target, train_classes, test_input, test_target, test_classes = dlc_practical_prologue.generate_pair_sets(1000)

In [42]:
# Weight-sharing "Siamese" LeNet
class Siamese(nn.Module):

    def __init__(self):
        super(Siamese, self).__init__()
        
        self.LeNet1 = nn.Sequential(
            nn.Conv2d(1,16,5),  # 16x10x10 (input is 1x14x14)
            nn.MaxPool2d(2),    # 16x5x5
            nn.ReLU(),
            nn.Conv2d(16,32,2), # 32x4x4
            nn.MaxPool2d(2),    # 32x2x2 (-> 1x128 before LeNet2)
            nn.ReLU()
        )
        self.LeNet2 = nn.Sequential(
            nn.Linear(128,64),  # 1x64
            nn.ReLU(),
            nn.Linear(64,32),   # 1x32
            nn.ReLU()
        )
        self.LeNet3 = nn.Sequential(
            nn.Linear(32,16),   # 1x16
            nn.Sigmoid(),
            nn.Linear(16,2)     # 1x2
        )
        
    def forward_bro(self, x):
        x = self.LeNet1(x)
        x = x.view(-1,1,128)
        x = self.LeNet2(x)
        return x
    
    def forward(self, x1, x2):
        x1 = self.forward_bro(x1)
        x2 = self.forward_bro(x2)
        x3 = x2 - x1
        x3 = self.LeNet3(x3)
        return x3

In [4]:
# control convolutions' dimensions
x1 = train_input.narrow(0,0,100)
x1 = x1[:,0].view(100,1,14,14)
weight1 = torch.empty(16,1,5,5).normal_()
bias1 = torch.empty(16).normal_()
x1 = F.conv2d(x1, weight1, bias1)

x1 = F.max_pool2d(x1, 2)
x1 = F.relu(x1)

weight2 = torch.empty(32,16,2,2).normal_()
bias2 = torch.empty(32).normal_()
x1 = F.conv2d(x1, weight2, bias2)

x1 = F.max_pool2d(x1, 2)
x1 = F.relu(x1)

x1 = x1.view(-1,1,128)
x1.shape

torch.Size([100, 1, 128])

In [5]:
def train_model(model, train_input, train_target, batch_size, nb_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)

    for e in range(nb_epochs):
        acc_loss = 0
        for b in range(0, train_input.size(0), batch_size):
            imgs = train_input.narrow(0, b, batch_size)
            imgs1 = imgs[:,0].view(batch_size, 1, 14, 14)
            imgs2 = imgs[:,1].view(batch_size, 1, 14, 14)
            output = model(imgs1, imgs2).view(batch_size, -1)
            loss = criterion(output, train_target.narrow(0, b, batch_size))
            acc_loss += loss.item()
            model.zero_grad()
            loss.backward()
            optimizer.step()
        print(e, acc_loss)

In [47]:
model = Siamese()
train_model(model, train_input, train_target, 100, 250)

0 6.123622298240662
1 4.720898121595383
2 4.099395543336868
3 3.461404412984848
4 3.139276295900345
5 3.180239260196686
6 2.565578520298004
7 2.4368543326854706
8 2.3552765399217606
9 1.9125934019684792
10 2.3408721536397934
11 1.9243998602032661
12 1.4963824972510338
13 1.6994921192526817
14 1.1543678119778633
15 0.9868257157504559
16 1.796875
17 2.146762691438198
18 1.070250265300274
19 0.7067676801234484
20 0.9381244853138924
21 1.1707879528403282
22 1.0643237680196762
23 0.8283866718411446
24 0.4535798504948616
25 0.397851618938148
26 0.30486978217959404
27 0.272602922283113
28 0.24062591511756182
29 0.2320688357576728
30 0.22679789271205664
31 0.222559267655015
32 0.2189282588660717
33 0.21520602330565453
34 0.2062004692852497
35 0.18021039571613073
36 0.17198903718963265
37 0.16893693478778005
38 0.16660790611058474
39 0.1646477752365172
40 0.16297075292095542
41 0.16151724103838205
42 0.16024419711902738
43 0.15911510167643428
44 0.158104807138443
45 0.1571925370953977
46 0.1563

In [7]:
def compute_nb_errors(model, input_data, target_data, batch_size):
    nb_errors = 0
    
    for b in range(0, input_data.size(0), batch_size):
            imgs = input_data.narrow(0, b, batch_size)
            target = target_data.narrow(0, b, batch_size)
            imgs1 = imgs[:,0].view(batch_size, 1, 14, 14)
            imgs2 = imgs[:,1].view(batch_size, 1, 14, 14)
            output = model(imgs1, imgs2).view(batch_size, -1)
            pred = output.max(1)[1]
            nb_errors += (pred-target).abs().sum().item()
    
    return nb_errors

In [51]:
train_errors = compute_nb_errors(model, train_input, train_target, 100)
test_errors = compute_nb_errors(model, test_input, test_target, 100)

In [52]:
train_errors/train_input.size(0)

0.002

In [55]:
test_errors/test_input.size(0)

0.165